In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import pandas as pd
import glob
import string
import nltk
import re


In [2]:
import os
import pandas as pd
import glob
import re


class Utils():
    root_directory = os.chdir('/content/drive/MyDrive/parser (1)')# input path
    list_of_files = glob.glob('**/*.txt', recursive=True)
    invalid_parsing = []
    trash = ['Волковыск-1.txt','Волковыск.txt','Вороново-1.txt',
         'Вороново.txt', 'Черновик-1.txt', 'Черновик.txt']
    for file in list_of_files:
        try:
             with open(file, encoding='utf-8') as f:
                    num = f.readline()
                    file_ = f.read()
                    if num and file_:
                        pass
        except:
            invalid_parsing.append(file)
    for i in invalid_parsing:
        list_of_files.remove(i)
    for i in trash: 
        list_of_files.remove(i)


list_of_files = Utils.list_of_files


plt_ = []
wbc_ = []
hgb_ = []
crea_ = []
ldh_ = []
sex_ = []
crp_ = []
tr = []
bir = []
ad = []
dis = []
ID_ = []
dgs_ = []
rf_ = []
alt_ = []
ast_ = []
pct_ = []

def ID(num):
  """returns ID of the patient"""
  for i in num.split():
    for char in i.split():
        if char.isnumeric():
            try:
                return int(char)
            except:
                return 'None'
            
            
def treatment(file_):
  """ returns srting of remedies used for the patient """
  for line in file_.split('\n'):
    if re.search('лечение', line):  # Does the same thing as "if 'hello' in line:"
        return(line.replace('Проведенное лечение:', \
                            '').replace('ЛФК', '').replace('ФТЛ', ''))
    

def born_adm_disch(file_):
    """ finds all nessesary dates in the epicrisis"""
    pattern = re.compile("(\d{2}).(\d{2}).(\d{4})") # check all dates
    birthday, admission = pattern.findall(file_)[:2]
    discharging = pattern.findall(file_)[-1]
    try:
        return ':'.join(birthday), ':'.join(admission), ':'.join(discharging)
    except:
        return 'None'


pattern_diagnosis = ['Диагноз:','Диагноз клинический:','Диагноз', 'Д-з:',
                     "Диагноз заключительный:"]
def diagnosis_dirty(file_):
  int_ = 0
  for i in file_.split():
    int_ += 1
    if i in pattern_diagnosis:
      return file_.split()[int_+1:int_+40]    
  

def crp(file_):
  """returns all crp values"""
  file_= ''.join(file_.split())
  file_ = re.sub(r'[ЦСC]РБ|\w\D(реактив.|реакт.|реак.)белок', 'С-реактивныйбелок', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'(?<!С-реактивныйбелок)\d\d\.\d\d', '', file_)
  file_ = re.sub(r',', '.', file_)
  pattern_1 = re.compile(r'(?:\w\Dреактивныйбелок|\w\Dреактивныйбелокдо)(\d*\.\d+|\d+)')
  pattern_2 = pattern_1.findall(file_)
  try:
    if pattern_2:
      return pattern_2
    else:
      return 'None'
  except:
    pass


def gender(file_):
  file_ = file_.title()
  file_ = ''.join(file_.split())
  file_ = re.sub(r'Диагноз.*', '', file_)
  patt1 = re.compile(r'[А-Я](\w{,19}(\w|\ич))[А-Я]\w{,20}(вна)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return 'female'
    else:
      return 'male'
  except:
    pass
  

def ldh(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wактатдегидрогеназ\w', 'лдг', file_)
  file_ = re.sub(r'(?<!лдг)\d\d\.\d\d', '', file_)
  patt1 = re.compile(r'(?<=лдг)\d{,4}')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass



def cre(file_):
  """Returns all creatinine values"""
  file_= ''.join(file_.split())  
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'(\wреатини\w|креатин|креат)(?![а-яА-Я,])', 'cre', file_)
  file_ = re.sub(r'(?<!cre)\d\d\.\d\d', '', file_)
  patt1 = re.compile(r'(?<=cre)\d{2,3}')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass


def hgb(file_):
  """returns list of str with level of hemoglobin"""
  file_= ''.join(file_.split())  
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  patt = re.compile(r'гемоглобин|гемогл|гб|гем')
  patt1 = re.sub(patt, r'hgb', file_)
  patt2 = re.compile(r'(?<=hgb)\d{2,3}')
  all_hgb = patt2.findall(patt1)
  try:
    if all_hgb:
      return all_hgb
    else:
      return 'None'
  except:
    pass


def wbc(file_):
  """returns list of str with level of wbc"""
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'10(\*|[еe])9', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\Dбщийанализкрови', 'оак', file_)
  file_ = re.sub(r'(?<=оак)гб\d*,', '', file_)
  file_ = re.sub(r',', '.', file_)
  patt1 = re.compile(r'(?<=оак|wbc)(?:л|лейкоцит\w)(\d*\.\d+|\d+)')
  all_wbc = patt1.findall(file_)
  try:
    if all_wbc:
      return all_wbc
    else:
      return 'None'
  except:
    pass


def plt(file_):
  """returns list of str with level of plt"""
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'10(\*|[еe])9', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  patt1 = re.compile(r'\wромбоцит\w|(?<![а-яА-Я])тр(?![а-яА-Я])')
  file_ = re.sub(patt1, r'plt', file_)
  file_ = re.sub(r'(?<!plt)\d\d\.\d\d', '', file_)
  patt2 = re.compile(r'(?<=plt)\d{3}')
  all_plt = patt2.findall(file_)
  try:
    if all_plt:
      return all_plt
    else:
      return 'None'
  except:
    pass


def rf(file_):
  file_= ''.join(file_.split())  
  file_ = re.sub(r'[а-яА-ЯёЁ]ДН[^Шш0-9]', 'ДН'.lower(), file_)
  file_ = re.sub(r'(?<=ДН)[1iI]{3}|[Шш]', '3', file_)
  file_ = re.sub(r'(?<=ДН)[1iI]{2}|1-2', '2', file_)
  file_ = re.sub(r'(?<=ДН)[оoОO]', '0', file_)
  patt1 = re.compile(r'(?<=ДН)\d')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 'None'  
  except:
      pass

def alt(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wланинаминотрансфераз\w|алат', 'алт', file_)
  file_ = re.sub(r'(?<!алт)\d\d\.\d\d', '', file_)
  file_ = re.sub(r',', '.', file_) 
  patt1 = re.compile(r'(?<=алт)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass


def ast(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wспартатаминотрансфераз\w|асат', 'аст', file_)
  file_ = re.sub(r'(?<!аст)\d\d\.\d\d', '', file_)
  file_ = re.sub(r',', '.', file_)
  patt1 = re.compile(r'(?<=аст)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass


def pct(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'(?<=\,\d)\,|\.\d{2}\.\d\d\.\d{2,4}.', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}(\.|)', '', file_)
  file_ = re.sub(r'[–-]|[():<>=]', '', file_)
  file_ = re.sub(r'менее', '', file_)
  file_ = re.sub(r'от', '', file_)
  file_ = re.sub(r'(\wрокальцитони\w|прокальцитон|прокальцит|прокальц|рст)(?![а-яА-Я,])', 'pct', file_)
  file_ = re.sub(r',', '.', file_)
  patt1 = re.compile(r'(?<=pct)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 'None'
  except:
    pass

In [3]:
for file in list_of_files:
    try:
         with open(file, encoding='utf-8') as f:
                num = f.readline()
                file_ = f.read()
                ID_.append(ID((num)))
                tr.append(treatment((file_)))
                bir.append(born_adm_disch(file_)[0])
                ad.append(born_adm_disch(file_)[1])
                dis.append(born_adm_disch(file_)[2])
                dgs_.append(diagnosis_dirty((file_)))
                crp_.append(crp(file_))
                sex_.append(gender(file_))
                ldh_.append(ldh(file_))
                crea_.append(cre(file_))
                hgb_.append(hgb(file_))
                wbc_.append(wbc(file_))
                plt_.append(plt(file_))
                rf_.append(rf(file_))
                alt_.append(alt(file_))
                ast_.append(ast(file_))
                pct_.append(pct(file_))
    except:
        print('stp')

In [ ]:
for i in pct_:
    if i != "None":
        print(i)

In [5]:
rfl = []
for o in rf_:
    if o:
        rfl.append(max(o))
    else:
        rfl.append('None')

In [6]:
diagnosis_cleaned =  []
for diagnos in dgs_:
    try:
        diagnosis_cleaned.append(' '.join(diagnos))
    except:
        diagnosis_cleaned.append('None')

In [7]:
len(diagnosis_cleaned)

2231

In [8]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [9]:
list_wo_punct = []
for sent in diagnosis_cleaned:
    try:
        sent = remove_punctuation(sent.lower())
        list_wo_punct.append(sent)
    except:
        pass

In [10]:
len(list_wo_punct)

2231

In [ ]:
# def coid19_finder(file_):
#     covid_patterns = ['covid19', 'коронавирусная', "короновирусная", "корон", "covid19", 'торсков2', 'sarscov2',"b342" ]
#     covid_list = []
#     for patt in covid_patterns:
#         for sent in file_:
#             pattern = re.sub(r'patt', 'covid', sent)
#             pattern_1 = re.compile(r'covid')
#             patt_2 = pattern_1.findall(''.join(pattern.split()))
#             try:
#                 covid_list.append(patt_2)
#             except:
#                 pass
#     c =[]
#     for i in covid_list:
#         if not i:
#             c.append(0)
#         else:
#             c.append(1)
#     return c

In [ ]:
#k = coid19_finder(list_wo_punct)

In [11]:
covid_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'covid19', 'covid', sent )
    pattern = re.sub(r'коронавирусная', 'covid', pattern)
    pattern = re.sub(r'короновирусная', 'covid', pattern)
    pattern = re.sub(r'b342', 'covid', pattern)
    pattern = re.sub(r'sarscov2', 'covid', pattern)
    pattern = re.sub(r'торсков2', 'covid', pattern)
    pattern_1 = re.compile(r'covid')
    patt_2 = re.search(pattern_1, pattern)
    #patt_2 = pattern_1.findall(''.join(pattern.split()))
    try:
        covid_list.append(patt_2)
    except:
        pass

In [12]:
c =[]
for i in covid_list:
    if not i:
        c.append(0)
    else:
        c.append(1)

In [13]:
pn_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'пневмония', 'J18', sent )
    pattern = re.sub(r'пне...ния', 'J18', pattern)
    pattern = re.sub(r'внегоспитальная', 'J18', pattern)
    pattern = re.sub(r'внебольничная', 'J18', pattern)
    #pattern = re.sub(r'sarscov2', 'covid', pattern)
    #pattern = re.sub(r'торсков2', 'covid', pattern)
    pattern_1 = re.compile(r'J18')
    patt_2 = re.search(pattern_1, pattern)
    #patt_2 = pattern_1.findall(''.join(pattern.split()))
    try:
        pn_list.append(patt_2)
    except:
        pass

In [14]:
p = []
for i in pn_list:
    if not i:
        p.append(0)
    else:
        p.append(1)

In [ ]:
# gr_list = []
# for sent in list_wo_punct:
#     pattern = re.sub(r'тяжёлая', 'тяж', sent )
#     pattern = re.sub(r'тяжелая', 'тяж', pattern)
#     pattern = re.sub(r'тяжёлое', '', pattern)
#     pattern = re.sub(r'внебольничная', 'J18', pattern)
#     #pattern = re.sub(r'sarscov2', 'covid', pattern)
#     #pattern = re.sub(r'торсков2', 'covid', pattern)
#     pattern_1 = re.compile(r'J18')
#     patt_2 = re.search(pattern_1, pattern)
#     #patt_2 = pattern_1.findall(''.join(pattern.split()))
#     try:
#         pn_list.append(patt_2)
#     except:
#         pass

In [27]:
gr_list = []
for sent in list_wo_punct:
  pattern = re.sub(r'среднетяж.л..', '', sent)
  pattern = re.sub(r'нетяж.л..', '', pattern)
  pattern_1 = re.compile(r'тяж.л..')
  pattern_2 = re.search(pattern_1, pattern)
  try:
    gr_list.append(pattern_2)
  except:
    pass

In [28]:
g = []
for i in gr_list:
    if not i:
        g.append(0)
    else:
        g.append(1)

In [29]:
dbt_list = []
for sent in list_wo_punct:
  sent = re.sub(r'тип', '', sent)
  pattern = re.compile(r'сахарныйдиабет|сд\d')
  pattern_1 = re.search(pattern, ''.join(sent.split()))
  try:
    dbt_list.append(pattern_1)
  except:
    pass
dbt = []
for i in dbt_list:
    if not i:
        dbt.append(0)
    else:
        dbt.append(1)    

In [30]:
chd_list = []
for sent in list_wo_punct:
  pattern = re.compile(r'ибс')
  pattern_1 = re.search(pattern, sent)
  try:
    chd_list.append(pattern_1)
  except:
    pass
chd = []
for i in chd_list:
    if not i:
        chd.append(0)
    else:
        chd.append(1)

In [31]:
obst_list = []
for sent in list_wo_punct:
  pattern = re.compile(r'ожирение')
  pattern_1 = re.search(pattern, sent)
  try:
    obst_list.append(pattern_1)
  except:
    pass
obst = []
for i in obst_list:
    if not i:
        obst.append(0)
    else:
        obst.append(1)

In [32]:
ba_list = []
for sent in list_wo_punct:
  pattern = re.compile(r'\sба\s')
  pattern_1 = re.search(pattern, sent)
  try:
    ba_list.append(pattern_1)
  except:
    pass
ba = []
for i in ba_list:
    if not i:
        ba.append(0)
    else:
        ba.append(1)

In [33]:
copd_list = []
for sent in list_wo_punct:
  pattern = re.compile(r'хобл')
  pattern_1 = re.search(pattern, sent)
  try:
    copd_list.append(pattern_1)
  except:
    pass
copd = []
for i in copd_list:
    if not i:
        copd.append(0)
    else:
        copd.append(1)

In [34]:
dfc={}
for i in g:
    dfc[i] = dfc.get(i, 0)+1
dfc

{0: 2017, 1: 214}

In [35]:
dfc={}
for i in p:
    dfc[i] = dfc.get(i, 0)+1
dfc

{0: 867, 1: 1364}

In [36]:
dfc={}
for i in c:
    dfc[i] = dfc.get(i, 0)+1
dfc

{0: 1951, 1: 280}

In [37]:
len(c)

2231

In [39]:
list_wo_punct[1]

'внегоспитальная пневмония в нд слева средней ст тяжести дн0 аг 1 ст риск3 н0 консультации врачейузких специалистов консилиумы врачей результаты инструментальных и аппаратных методов исследования вид исследования rграфия огк2 проекции ээд 042 мзв регистр № 25367 при сравнении с'

In [ ]:
# data_dct = {'Platlets':plt_, 'Leucocytes': wbc_, 'Hemoglobin': hgb_,
#           'Creatinine': crea_, 'LDG':ldh_, 'Gender':sex_, 'CRP':crp_,
#            'Treatment':tr, 'Birthday': bir, 'Admission':ad,
#            'Discharge':dis, 'ID':ID_, 'Diagnosis': list_wo_punct,
#            'COVID-19': c, 'Pneumonia': p, 'Grade': g, 'RF': rfl, 'ALT': alt_,
#            'AST': ast_, 'Procalcitonine': pct_ }

In [40]:
data_dct = {'Platlets':plt_, 'Leucocytes': wbc_, 'Hemoglobin': hgb_,
           'Creatinine': crea_, 'LDG':ldh_, 'Gender':sex_, 'CRP':crp_,
            'Treatment':tr, 'Birthday': bir, 'Admission':ad,
            'Discharge':dis, 'ID':ID_, 'Diagnosis': list_wo_punct,
            'COVID-19': c, 'Pneumonia': p, 'Grade': g, 'RF': rfl, 'ALT': alt_,
            'AST': ast_, 'Procalcitonine': pct_, 'Diabetes': dbt, 'CHD': chd, 'Obesity': obst, 'BA': ba, 'COPD': copd }

In [41]:
dataFrame = pd.DataFrame(data_dct)

In [42]:
dataFrame.head(5)

,Platlets,Leucocytes,Hemoglobin,Creatinine,LDG,Gender,CRP,Treatment,Birthday,Admission,Discharge,ID,Diagnosis,COVID-19,Pneumonia,Grade,RF,ALT,AST,Procalcitonine,Diabetes,CHD,Obesity,BA,COPD
0,"[268, 317, 344]","[12.88, 4.68, 7.81]","[127, 133, 141]",[56],None,female,"[48, 12, 6]","лефлокс, амброксол, индап, амлодипин . аспикар...",29:10:1949,22:05:2017,30:05:2017,3436.0,острый бронхит дн0 ибс атеросклеротический кар...,0,0,0,0,[17],[23],None,0,1,0,0,0
1,"[187, 274, 274]","[8.31, 8.29, 8.29]","[143, 138, 138]",None,None,male,[50.6],"цефтриаксон , флустоп , амброксол , ФТ ,",29:01:1970,31:12:2019,10:01:2020,7826.0,внегоспитальная пневмония в нд слева средней с...,0,1,0,0,[126],[56],None,0,0,0,0,0
2,"[212, 316]","[20.5, 9.7]","[128, 143]",None,None,female,"[256.5, 35.9, 14.02]","цефтриаксон, левофлоксацин, амброксол,клопидог...",15:11:1960,02:02:2020,14:02:2020,743.0,j18 пневмония без уточнения возбудителя основн...,0,1,0,2,[18],[16],None,0,0,0,1,1
3,"[335, 460]","[6.99, 7.87]","[144, 104]",[66],None,male,None,"цефтриаксон . амброксол , аспикард , касарк ...",28:10:1943,10:03:2020,19:03:2020,1603.0,j18 пневмония без уточнения возбудителя основн...,0,1,0,1,None,None,None,0,1,0,0,0
4,[255],[7.55],[149],[104],None,male,[31.5],"цефтриаксон . амброксол . флустоп , ФТ ,",13:10:1995,15:01:2020,23:01:2020,325.0,внегоспитальная пневмония в вд слева средней с...,0,1,0,0,[63],[28],None,0,0,0,0,0


In [43]:
dataFrame[dataFrame['ID'] == 635.0]

,Platlets,Leucocytes,Hemoglobin,Creatinine,LDG,Gender,CRP,Treatment,Birthday,Admission,Discharge,ID,Diagnosis,COVID-19,Pneumonia,Grade,RF,ALT,AST,Procalcitonine,Diabetes,CHD,Obesity,BA,COPD
411,"[364, 191]","[4.56, 8.27]","[123, 155]",None,None,male,[7.7],"левофлоксацин , диклофенак , амброксол , омеп...",05:06:1981,28:01:2020,07:02:2020,635.0,внегоспитальная плевропневмония в нд справа в ...,0,1,0,1,[11],[10],None,0,0,0,0,0


In [44]:
len(dataFrame['CRP'])

2231

In [45]:
dataFrame.to_csv('data_3.csv')